In [31]:
import geopandas as gpd
import random
import requests

In [2]:
bangalore_b = gpd.read_file("/home/lohitd@nextbillion.ai/Downloads/bangalore_b_nominatim.geojson")
bangalore_tomtom2 = gpd.read_file("/home/lohitd@nextbillion.ai/Downloads/bangalore-tomtom2.geojson")

In [3]:
bangalore_tomtom2.columns

Index(['junction_id', 'highway', 'maxspeed', 'name', 'name:en', 'oneway',
       'tomtom_id', 'FOW', 'FRC', 'routing_class', 'access', 'hgv', 'junction',
       'hgv:maxwidth', 'hgv:maxheight', 'accurate_location', 'toll_gate',
       'hgv:maxweight', 'toll', 'geometry'],
      dtype='object')

In [4]:
bangalore_tomtom2.head(3)

,junction_id,highway,maxspeed,name,name:en,oneway,tomtom_id,FOW,FRC,routing_class,access,hgv,junction,hgv:maxwidth,hgv:maxheight,accurate_location,toll_gate,hgv:maxweight,toll,geometry
0,e5941290-1b35-496b-9876-89373807e84d,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (77.68150 13.03980)
1,00004e32-3000-2800-ffff-ffffce218243,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (77.53020 12.98778)
2,00004e32-3000-2800-ffff-ffffcce8bdea,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (77.64519 13.01042)


In [5]:
access_no = bangalore_tomtom2[bangalore_tomtom2['access'] == "no"]

In [6]:
access_no['centroids'] = access_no['geometry'].centroid

<ipython-input-6-0d46ed49394e>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  access_no['centroids'] = access_no['geometry'].centroid
/home/lohitd@nextbillion.ai/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [22]:
access_no.head(3)

,junction_id,highway,maxspeed,name,name:en,oneway,tomtom_id,FOW,FRC,routing_class,...,toll_gate,hgv:maxweight,toll,geometry,centroids,origin,destination,centroid_lon,centroid_lat,centroid_coordinates
4,None,footway,0,None,None,no,00004e32-3000-0400-0000-00000366b689,20,other_road,7,...,None,None,None,"LINESTRING (77.65123 12.89868, 77.65087 12.898...",POINT (77.65092 12.89875),"13.09768197216262,77.5380340307321","13.058122429679441,77.66984574054248",77.650924,12.898755,"12.898754630755045,77.65092352777764"
6,None,residential,45,None,None,no,00004e32-3000-0400-0000-0000022a5e8b,3,local_road_of_minor_importance,7,...,None,None,None,"LINESTRING (77.70092 12.98279, 77.70102 12.982...",POINT (77.70099 12.98288),"13.072151356590355,77.48630928801968","13.101914795679653,77.58302564455917",77.700986,12.982883,"12.982883479563865,77.70098586547103"
18,None,residential,0,None,None,no,d5d51d05-a7a0-48b4-af58-69c942826d91,3,local_road_of_minor_importance,7,...,None,None,None,"LINESTRING (77.61531 13.12461, 77.61560 13.125...",POINT (77.61513 13.12566),"12.914939198091588,77.71267735425485","12.969396741254881,77.60104192034113",77.615129,13.125664,"13.125663764473893,77.6151290902805"


In [12]:
#creating bounds
boundary_bounds = bangalore_b.bounds
minx = boundary_bounds.loc[0,'minx']
miny = boundary_bounds.loc[0,'miny']
maxx = boundary_bounds.loc[0,'maxx']
maxy = boundary_bounds.loc[0,'maxy']

#create random coordinates
coordinates = []
for each in range(61028):
    lng = random.uniform(minx, maxx)
    lat = random.uniform(miny, maxy)
    coordinates.append(f"{lat},{lng}")
    
#split coordinates as origin and destination
origin = coordinates[0:int(len(coordinates)/2)]
destination = coordinates[int(len(coordinates)/2):int(len(coordinates)+1)]

In [14]:
access_no['origin'] = origin
access_no['destination'] = destination

/home/lohitd@nextbillion.ai/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [18]:
access_no['centroid_lon'] = access_no.centroids.x
access_no['centroid_lat'] = access_no.centroids.y

/home/lohitd@nextbillion.ai/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [20]:
access_no['centroid_coordinates'] = access_no['centroid_lat'].astype(str) + "," + access_no['centroid_lon'].astype(str)

In [28]:
access_no.to_csv('/home/lohitd@nextbillion.ai/banglore_access_no_line_centroids.csv', index=False)

In [ ]:
'https://compass.nextbillion.io/directions?origin=12.9727462,77.7129297&dest=12.95172596,77.64041625&waypoints=12.9718376,77.7026649&waypoints=12.9727677,77.6991394&waypoints=12.94827060,77.64615213&approaches=unrestricted&approaches=unrestricted&approaches=unrestricted&approaches=unrestricted&approaches=unrestricted&travelMode=4&departureTime=1654065622343&alternative=false&serviceProviders=Plain%20NB&session=IN7Fz1&alternativeNum=1&annotation=false'

In [29]:
nb_request_links = []

for index, row in access_no.iterrows():
    nb_request_links.append(f"https://api.nextbillion.io/directions/json?origin={row['origin']}&destination={row['destination']}&session=c0sinx&alternatives=false&altcount=1&annotations=false&approaches=unrestricted;unrestricted&avoid=&mode=4w&waypoints={row['centroid_coordinates']}&departure_time=1654598935&key=plaintesting")
    
access_no['nb_request_links'] = nb_request_links

/home/lohitd@nextbillion.ai/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
nb_responses = []

for link in nb_request_links:
    nb_response = requests.get(link)
    nb_responses.append(nb_response.text)

In [ ]:
access_no['nb_responses'] = pd.Series(nb_responses)

In [ ]:
asahi_marker_links = []

for index, row in asahi.iterrows():
    asahi_marker_links.append(f"L.marker([{row['lat']}, {row['lon']}], {{icon: asahiIcon}}).addTo(map).bindPopup('Brand:{row['brand']}<br>Vending:{row['vending']}<br>Nearby:{row['nearby']}<br>Landmark:{row['landmark']}<br><a href = {row['google_sv']} >Google Street View</a>');")
    
asahi['markerlink'] = asahi_marker_links